In [1]:
import numpy as np
import matplotlib as plt
import time
from tqdm import *


In [2]:
import torch
from torchvision import datasets
from torchvision import transforms
from torch.utils.data import DataLoader
import torch.optim as optim

In [3]:
batch_size=64
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.1307,),(0.3081,))
])
train_dataset=datasets.MNIST(root='/Users/steve/Desktop',
                            train=True,
                            download=True,
                            transform=transform)
train_loader=DataLoader(train_dataset,
                        shuffle=True,
                        batch_size=batch_size
)
test_dataset=datasets.MNIST(root='/Users/steve/Desktop',
                           train=False,
                           download=True,
                        transform=transform
)
test_loader=DataLoader(test_dataset,
                       shuffle=True,
                       batch_size=batch_size
)



In [4]:
class CNN(torch.nn.Module):
    def __init__(self):
        super(CNN,self).__init__()
        self.conv1 = torch.nn.Conv2d(1,10,kernel_size=5)
        self.conv2= torch.nn.Conv2d(10,20,kernel_size=5)
        #卷积核定义
        self.pooling=torch.nn.MaxPool2d(2)
        #由于不含权重，pooling层只需要定义一次
        self.linear1=torch.nn.Linear(320,120)
        self.linear2=torch.nn.Linear(120,10)
        self.relu=torch.nn.functional.relu
        #relu在多分类器层中
    def forward(self,x):
        batch_size=x.size(0)
        #由于在batch拆分时不能达到完美切割，最后一个batch的大小与其他batch可能不同，所以要重新定义
        x=self.relu(self.pooling(self.conv1(x)))
        x=self.relu(self.pooling(self.conv2(x)))
        #一般是先pooling后relu
        x=x.view(batch_size,-1)
        #x.view的结果是（batch_size，处理后图像的C*H*W）
        x=self.linear2(self.relu(self.linear1(x)))
        #最后一层分类后不需要激活函数，因为求交叉熵时用的CrossEntropyLoss里面包含softmax步骤
        return x
Model=CNN()

In [5]:
criterion = torch.nn.CrossEntropyLoss(reduction='mean')
optimizer = optim.SGD(Model.parameters(), lr=0.01,momentum=0.5)

In [6]:
def train(epoch):
    running_loss = 0.0
    for batch_idx, data in enumerate(train_loader, 0):
        inputs, target = data
        optimizer.zero_grad()
 
        outputs = Model(inputs)
        loss = criterion(outputs, target)
        loss.backward()
        optimizer.step()
 
        running_loss += loss.item()
        if batch_idx % 300 == 299:
            print('[%d, %5d] loss: %.3f' % (epoch+1, batch_idx+1, running_loss/300))
            running_loss = 0.0
 
 
def test():
    correct = 0
    total = 0
    with torch.no_grad():
        for data in test_loader:
            images, labels = data
            outputs = Model(images)
            _, predicted = torch.max(outputs.data, dim=1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    print('accuracy on test set: %d %% ' % (100*correct/total))
 
 
if __name__ == '__main__':
    for epoch in range(10):
        train(epoch)
        test()


[1,   300] loss: 1.006
[1,   600] loss: 0.232
[1,   900] loss: 0.172
accuracy on test set: 95 % 
[2,   300] loss: 0.124
[2,   600] loss: 0.112
[2,   900] loss: 0.093
accuracy on test set: 97 % 
[3,   300] loss: 0.079
[3,   600] loss: 0.077
[3,   900] loss: 0.072
accuracy on test set: 98 % 
[4,   300] loss: 0.061
[4,   600] loss: 0.065
[4,   900] loss: 0.056
accuracy on test set: 97 % 
[5,   300] loss: 0.052
[5,   600] loss: 0.053
[5,   900] loss: 0.045
accuracy on test set: 98 % 
[6,   300] loss: 0.044
[6,   600] loss: 0.043
[6,   900] loss: 0.042
accuracy on test set: 98 % 
[7,   300] loss: 0.039
[7,   600] loss: 0.038
[7,   900] loss: 0.038
accuracy on test set: 98 % 
[8,   300] loss: 0.033
[8,   600] loss: 0.035
[8,   900] loss: 0.033
accuracy on test set: 98 % 
[9,   300] loss: 0.031
[9,   600] loss: 0.030
[9,   900] loss: 0.033
accuracy on test set: 98 % 
[10,   300] loss: 0.027
[10,   600] loss: 0.027
[10,   900] loss: 0.028
accuracy on test set: 98 % 
